In [77]:
import git
import time
import pickle
from tinyec import registry
from Crypto.Cipher import AES
import hashlib, secrets, binascii

In [78]:
def decrypt_AES_GCM(ciphertext, nonce, authTag, secretKey):
    aesCipher = AES.new(secretKey, AES.MODE_GCM, nonce)
    plaintext = aesCipher.decrypt_and_verify(ciphertext, authTag)
    return plaintext

def ecc_point_to_256_bit_key(point):
    sha = hashlib.sha256(int.to_bytes(point.x, 32, 'big'))
    sha.update(int.to_bytes(point.y, 32, 'big'))
    return sha.digest()

curve = registry.get_curve('brainpoolP256r1')

def decrypt_ECC(encryptedMsg, privKey):
    (ciphertext, nonce, authTag, ciphertextPubKey) = encryptedMsg
    sharedECCKey = privKey * ciphertextPubKey
    secretKey = ecc_point_to_256_bit_key(sharedECCKey)
    plaintext = decrypt_AES_GCM(ciphertext, nonce, authTag, secretKey)
    return plaintext

In [82]:
repo = git.Repo("/Users/Chris-OSX/Desktop/TEL252_Central_de_datos")

In [95]:
while (True):
    repo.remotes.origin.pull()
    privKey = secrets.randbelow(curve.field.n) #b
    pubKeyB = privKey * curve.g #B
    
    with open('pub_keyB.pickle', 'wb') as file_pubKeyB:
        pickle.dump(pubKeyB, file_pubKeyB)
        
    repo.git.add(repo.working_dir)
    commit_output = repo.git.commit(m="Llave pub Update")
    push_output = repo.git.push('--set-upstream', repo.remote().name, "main")
        
    time.sleep(7)
    repo.remotes.origin.pull()

    with open('encrypt_data.pickle', 'rb') as file_encrypt_data:
        encryptedMsg = pickle.load(file_encrypt_data) #B
        
    encryptedMsgObj = {
    'ciphertext': binascii.hexlify(encryptedMsg[0]),
    'nonce': binascii.hexlify(encryptedMsg[1]),
    'authTag': binascii.hexlify(encryptedMsg[2]),
    'ciphertextPubKey': hex(encryptedMsg[3].x) + hex(encryptedMsg[3].y % 2)[2:]
    }
    decryptedMsg = decrypt_ECC(encryptedMsg, privKey)
    print("decrypted msg:", decryptedMsg.decode('UTF-8'))

BrokenPipeError: [Errno 32] Broken pipe

In [ ]:
repo.remotes.origin.pull()